# COGS 108 - Final Project 

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that PIDs will be scraped from the public submission, but student names will be included.)

* [  ] YES - make available
* [  ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Toland (Venti)
- Chongyun (Daiki)
- Zhaoge Ouyang
- cw

# Group Members IDs

- A15747306(Kyle)
- A14136586
- A15405768 (Zhaoge)
- A######## (Cary)

# Research Question

We plan to look at games and their sales data to figure out how does a game's genre, merchandise, critic score, online/total players count, player base (gaming platform) affect a games popularity and sales? We want to know which factors contribute to the highest sales for gaming platform that are from 2000 onwards.

## Background and Prior Work

This question is interesting because:
We will be able to see the current trends in gaming.
When a game is released by a company, the critic score, the company its made by, and the genre can predict how well the game does in the market. These all influence the popularity of a game. So according to these variables, we want to predict how popular a game is and if it is worth playing. Recently some popular games such as among us and Genshin impact have skyrocketed in popularity and sales, so we want to investigate what exactly makes games like this popular so fast.
References (include links):
- 1)https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.549.6080&rep=rep1&type=pdf
- 2)https://www.kaggle.com/gulsahdemiryurek/video-game-sales-exploratory-data-analysis

# Hypothesis


Our hypothesis is that games that fall into the adventure and strategy categories to have the highest sales and rating/rank.

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: Metacritic Games Stats 2011-2019
- Link to the dataset: https://www.kaggle.com/skateddu/metacritic-games-stats-20112019?select=metacritic_games.csv
- Number of observations: 4018

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: Video Game Sales
- Link to the dataset: https://www.kaggle.com/gregorut/videogamesales
- Number of observations: 11493

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: IGN Games of 20 Years
- Link to the dataset: https://raw.githubusercontent.com/john7obed/ign_games_of_20_years/master/ign.csv
- Number of observations: 18624

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

Let's begin by importing some packages for analysis.

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

In [ ]:
# Configure libraries
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Don't display too many rows/cols of DataFrames
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

Let's also import the 3 datasets listed for the analysis.

In [ ]:
metacritic = pd.read_csv('metacritic_games.csv')
ign = pd.read_csv('ign.csv', index_col='Unnamed: 0')
vgsales = pd.read_csv('vgsales_lessNaN.csv')

To make sure that our data is imported properly:

In [ ]:
metacritic.head()

In [ ]:
ign.head()

In [ ]:
vgsales.head()

# Data Cleaning

Describe your data cleaning steps here.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

We would like to merge the three DataFrames into one. First, we take a look at the columns of each DataFrame.

In [ ]:
vgsales.columns

In the datasets, same things are labelled differently, such as name of a game. Let's standardize the column labels of each entry before we do any data manipulation.

In [ ]:
# vgsales column names should be all lowercase
vgsales = vgsales.rename(columns=str.lower)

# misc renames
metacritic = metacritic.rename(columns={'game': 'name'})
ign = ign.rename(columns={'title':'name'})
vgsales = vgsales.rename(columns={'year':'release_year', 'na_sales' : 'na_sales_million', 'pal_sales': 'pal_sales_million', 'jp_sales': 'jp_sales_million', 'other_sales': 'other_sales_million', 'global_sales': 'global_sales_million'})

There are some columns that are for sure irrelevant to the analysis, such as the URL path of a game review on IGN.

In [ ]:
ign = ign.drop(columns='url')
vgsales = vgsales.drop(columns=['last_update', 'url', 'status', 'vgchartzscore', 'img_url'])

In all datasets, although certain games may have same titles, they are treated as different observations because they may be in different platforms. Let's standardize our titles this way as well.

In [ ]:
vgsales['platform'].unique()

In [ ]:
ign['platform'].unique()

In [ ]:
metacritic['platform'].unique()

Let's try to standardize a subset of the platforms.

In [ ]:
def standardize_platform(string):
    platforms = ['Wii', 'NES', 'PC', 'GB', 'DS', 'X360', 'SNES', 'PS3', 'PS4',
       '3DS', 'PS2', 'GBA', 'NS', 'N64', 'PS', 'XOne', 'WiiU',
       'XB', 'PSP', 'GC', 'GBC', 'PSV']
    # I did not include: Macintosh, Lynx, Saturn, NeoGeo Pocket Color
    # Game.Com, Dreamcast, Dreamcast VMU, WonderSwan, Arcade,Nintendo 64DD,
    # WonderSwan Color, DVD / HD Video Game, Wireless, Pocket PC, N-Gage,
    # iPod, Genesis, TurboGrafx-16, NeoGeo, ...everything down there except 3DS and XBox One
    
    if string in ['PlayStation Vita', 'VITA']:
        return 'PSV'
    elif string == 'Xbox 360':
        return 'X360'
    elif string == 'PlayStation 3':
        return 'PS3'
    elif string == 'PC':
        return 'PC'
    elif string == 'Nintendo DS':
        return 'DS'
    elif string in ['Nintendo 3DS', 'New Nintendo 3DS', '3DS']:
        return '3DS'
    elif string == 'Wii':
        return 'Wii'
    elif string in ['PlayStation 4', 'PS4']:
        return 'PS4'
    elif string in ['Wii U', 'WIIU']:
        return 'WiiU'
    elif string == 'PlayStation Portable':
        return 'PSP'
    elif string == 'PlayStation':
        return 'PS'
    elif string == 'Nintendo 64':
        return 'N64'
    elif string == 'Game Boy':
        return 'GB'
    elif string == 'Game Boy Color':
        return 'GBC'
    elif string == 'PlayStation 2':
        return 'PS2'
    elif string == 'Game Boy Advance':
        return 'GBA'
    elif string == 'Xbox':
        return 'XB'
    elif string == 'GameCube':
        return 'GC'
    elif string == 'NES':
        return 'NES'
    elif string == 'Super NES':
        return 'SNES'
    elif string in ['Xbox One', 'XONE']:
        return 'XOne'
    elif string == 'Switch':
        return 'NS'
    elif string not in platforms:
        return np.nan
    else:
        return string

Apply the standardize function to each dataset and check the unique values again.

In [ ]:
metacritic['platform'] = metacritic['platform'].apply(standardize_platform)
vgsales['platform'] = vgsales['platform'].apply(standardize_platform)
ign['platform'] = ign['platform'].apply(standardize_platform)
vgsales.dropna(subset=['platform'], inplace=True)
ign.dropna(subset=['platform'], inplace=True)
metacritic.dropna(subset=['platform'], inplace=True)

In [ ]:
vgsales['platform'].unique()

In [ ]:
vgsales[vgsales['name'].str.startswith('Fire Emblem')]

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*